In [1]:
!pip install pdfplumber
!pip install fpdf
!pip install deep_translator
!pip install nltk
!python -m nltk.downloader popular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256=d6db32a51637ff0c52c85193c8f3b6862791e0c38e1687ceafc81639f3303a26
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_d

In [6]:
from deep_translator import GoogleTranslator
from nltk.tokenize import sent_tokenize
from fpdf import FPDF
import pdfplumber

## **Extract the Text from the File**

In [3]:
def extract(page):
  """Extract PDF text and Delete in-paragraph line breaks."""
  # Get text
  extracted = page.extract_text()
  # Delete in-paragraph line breaks
  extracted = extracted.replace(".\n", "**/m" # keep par breaks
                      ).replace(". \n", "**/m" # keep par breaks
                      ).replace("\n", "" # delete in-par breaks
                      ).replace("**/m", ".\n\n") # restore par break
  return extracted

## **Translation**

In [10]:
def translate_extracted(Extracted):
  """Wrapper for Google Translate with upload workaround."""
  # Set-up and wrap translation client
  translate = GoogleTranslator(source='auto', target='nl').translate
  # Split input text into a list of sentences
  sentences = sent_tokenize(Extracted)
  # Initialize containers
  translated_text = ''
  source_text_chunk = ''
  # collect chuncks of sentences, translate individually
  for sentence in sentences:
    # if chunck + current sentence < limit, add the sentence
    if ((len(sentence.encode('utf-8')) +  len(source_text_chunk.encode('utf-8')) < 5000)):
      source_text_chunk += ' ' + sentence
    # else translate chunck and start new one with current sentence
    else:
      translated_text += ' ' + translate(source_text_chunk)
     # if current sentence smaller than 5000 chars, start new chunck
    if (len(sentence.encode('utf-8')) < 5000):
       source_text_chunk = sentence
     # else, replace sentence with notification message
    else:
       message = "<<Omitted Word longer than 5000bytes>>"
       translated_text += ' ' + translate(message)
       # Re-set text container to empty
       source_text_chunk = ''
  # Translate the final chunk of input text, if there is any valid   text left to translate
  if translate(source_text_chunk) != None:
    translated_text += ' ' + translate(source_text_chunk)
  return translated_text

In [11]:
# Open PDF
with pdfplumber.open("/content/CV.pdf") as pdf:
  # Treat each page individually
  for page in pdf.pages:
    # Extract Page
    extracted = extract(page)
    # Translate Page
    if extracted != "":
      # Translate paragraphs individually to keep breaks
      paragraphs = extracted.split("\n\n")
      translated = "\n\n".join(
        [translate_extracted(paragraph) for paragraph in paragraphs]
        )
    else:
      translated = extracted

## **Output**

In [12]:
# # Write the extracted and translated text to PDF file
fpdf = FPDF()
fpdf.set_font("Helvetica", size = 7)
fpdf.add_page()
fpdf.multi_cell(w=0, h=5,
                txt= translated.encode("latin-1",
                                      errors = "replace"
                              ).decode("latin-1"))
# Save all FPDF pages
fpdf.output("/content/output.pdf")

''

In [13]:
# Write the extracted and translated text to the output text file
with open("/content/output.txt", "w") as file:
    file.write(translated)